In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('sec__edgar_company_info.csv')

In [3]:
new_df = []
for i in df['Company Name']:
    clean = re.sub('[^A-Za-z]+', ' ', i)
    wp = clean.lstrip()
    new_df.append(wp)

In [4]:
dt = dict(zip(new_df,df['Company CIK Key']))

In [5]:
dt

{'J INC': 1438823,
 'A LIFESAFER HOLDINGS INC ': 1509607,
 'ARIZONA DISCOUNT PROPERTIES LLC': 1457512,
 'PAINTBALL CORP': 1433777,
 'LLC': 1173062,
 'S MEDIA GROUP LLC': 1447162,
 'TV COMMUNICATIONS INC ': 1479357,
 'MKTG INC ': 886475,
 'OHANA LABS INC ': 1703629,
 'OURCROWD INVESTMENT IN MST L P ': 1599496,
 'Y Z QUEENCO LTD ': 1623088,
 'CLUB DOMAINS LLC': 1577787,
 'SMILE COMMUNICATIONS LTD': 1402606,
 'MEDTECH INC': 800181,
 'PHARMA INC ': 1307969,
 'DIESEL CORP': 1414080,
 'ENTERTAINMENT GROUP INC': 1322914,
 'ASSET MANAGEMENT LLC ': 1114831,
 'AFWAIN COURITY KATHY LLC': 1398981,
 'CAT THIEF GOLD IN MY STARS LLC': 1406134,
 'DIXIE UNION UNDER FIRE LLC': 1395687,
 'EL CORREDOR ONE BAD CAT LLC': 1398457,
 'FREUD LAVISH NUMBERS LLC': 1402813,
 'GOLD CASE JETAZELLE LLC': 1398456,
 'HOLD THAT TIGER TOJUR LLC': 1394816,
 'INDIAN CHARLIE HELLO KITTY KITTY LLC': 1395686,
 'KAFWAIN JOVIAL WINGS LLC': 1399020,
 'LIDO PALACEMADA MEADOWLAKE LLC': 1398982,
 'MACHO UNO GO DONNA GO LLC': 139481

In [6]:
def obtain_cik():
    d = str(input("Enter what you want: ")).upper()
    CIK_code = [usinp + " : " + str(dt[usinp]) for usinp in dt.keys() if usinp.startswith(d)]
    conf_cik = CIK_code[0]    
    print(conf_cik + '\n\n' + "Other suggestions: \n" + str(CIK_code[1:]))  
    return conf_cik

In [7]:
fin_cik = obtain_cik()

Enter what you want: apple inc
APPLE INC : 320193

Other suggestions: 
[]


In [8]:
fin_cik

'APPLE INC : 320193'

In [9]:
CIK = re.sub('\D', '', fin_cik)

In [10]:
URL = "https://www.sec.gov/edgar/browse/?CIK=" + str(CIK) + "&owner=exclude"

In [11]:
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time


In [12]:
Path = "C:/Program Files (x86)/chromedriver.exe"
driver = webdriver.Chrome(Path)
driver.get(URL)
button = driver.find_element_by_class_name("btn.btn-sm.btn-info")

try:    
    if button.is_displayed():
        button.click()
        element = WebDriverWait(driver,15).until(
        EC.presence_of_element_located((By.LINK_TEXT, "Annual report [Section 13 and 15(d), not S-K Item 405]")))
        risk_url = element.get_attribute('href')

        
    elif not button.is_displayed():
        element = WebDriverWait(driver,15).until(
        EC.presence_of_element_located((By.LINK_TEXT, "Annual report [Section 13 and 15(d), not S-K Item 405]")))
        risk_url = element.get_attribute('href')               

except:
    print('Either the company does not exists or the 10K fillings is not available. Please try again')
finally:
    driver.quit()    

In [13]:
risk_url

'https://www.sec.gov/ix?doc=/Archives/edgar/data/0000320193/000032019320000096/aapl-20200926.htm'

In [14]:
new_driv = webdriver.Chrome(Path)
new_driv.get(risk_url)
page_text = new_driv.find_element_by_id('dynamic-xbrl-form').text
new_driv.quit()

In [ ]:
start = page_text.find("Item 1A.    Risk Factors") + len("Item 1A.    Risk Factors")

end = page_text.find("Item 1B.    Unresolved Staff Comments")

substring = page_text[start:end]

In [ ]:
with open('10-K.txt', "w", encoding="utf-8") as f:
    f.write(page_text)

In [41]:
result = re.search(('Item 1A.|ITEM 1A.)(\s)*(Risk|RISK)(.+)(\n)+(.+)(\n)+(Item 1B.|ITEM 1B.)(\s)*([^\n])(Unresolved Staff Comments)', page_text)
print(result.group(1))

SyntaxError: invalid syntax (<ipython-input-41-b22910b168de>, line 2)

In [38]:
page_text

'UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\nFORM 10-K\n(Mark One)\n☒ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the fiscal year ended September 26, 2020\nor\n☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the transition period from              to             .\nCommission File Number: 001-36743\nApple Inc.\n(Exact name of Registrant as specified in its charter)\nCalifornia 94-2404110\n(State or other jurisdiction\nof incorporation or organization)\n(I.R.S. Employer Identification No.)\nOne Apple Park Way\nCupertino, California\n95014\n(Address of principal executive offices) (Zip Code)\n(408) 996-1010\n(Registrant’s telephone number, including area code)\nSecurities registered pursuant to Section 12(b) of the Act:\nTitle of each class\nTrading symbol(s)\nName of each exchange on which registered\nCommon Stock, $0.00001 par value per share\nAAPL\nThe Nasdaq St